In [ ]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.results as res
from biogeme.expressions import Beta, DefineVariable
import math

In [ ]:
networks = ['18h', '18u', '27', '36']
inter_times = [1, 5, 8]
apriori_probs = [2]

indicator = {}
scenarios = {}
for n in networks:
    for t in inter_times:
        for p in apriori_probs:
            scenarios['{}:{}:{}'.format(n,t,p)] = 0
            indicator['{}:{}:{}'.format(n,t,p)] = 0

In [ ]:
def bio_choice(row, betas, with_data):
    atribs = ['time', 'n_nodes', 'angular_cost']
    utils = []
    for i in range(0,with_data+1):
        if row['av_{}'.format(i)] == 1:
            u = 0
            for a in atribs:
                u += betas['B_{}'.format(a)]*row['{}_{}'.format(a,i)]
            utils.append(u)
        else:
            break
    
    enums = []
    for i in utils:
        enums.append(math.exp(i))
    denom = sum(enums)
    
    probs = []
    for enum in enums:
        probs.append(enum/denom)
    
    print(probs)
    max_prob = max(probs)
    return pd.Series([probs.index(max_prob),max_prob])

In [ ]:
def tes_choice(row, with_data):
    probs = []
    for i in range(0,with_data+1):
        probs.append(row['tesis_prob_{}'.format(i)])
    max_prob = max(probs)
    return pd.Series([probs.index(max_prob),max_prob])

In [ ]:
for e in scenarios:
    
    # Reading DB
    bd_bio = pd.read_csv('no_infs/ninf_{}.csv'.format(e))
    bd_calc = pd.read_csv('infs/inf_{}.csv'.format(e))
    columns = ['tesis_prob_{}', 'n_nodes_{}', 'time_{}', 'distance_{}', 'angular_cost_{}']
    print('    Database read')
    
    # Cleaning Zero columns
    with_data = 0
    while bd_bio['time_{}'.format(with_data)].count() > 0:
        with_data += 1
        if 'time_{}'.format(with_data) in bd_bio.columns:
            continue
        else:
            break
    with_data -= 1
    bd_bio = bd_bio.iloc[ : , 0:5*(with_data+1)]
    print('    Zeros deleted in bio')
    
    # Cleaning Zero columns
    with_data2 = 0
    while bd_calc['time_{}'.format(with_data2)].count() > 0:
        with_data2 += 1
        if 'time_{}'.format(with_data2) in bd_calc.columns:
            continue
        else:
            break
    with_data2 -= 1
    bd_calc = bd_calc.iloc[ : , 0:5*(with_data2+1)]
    print('    Zeros deleted in calc')
    
    # Adding choice column
    bd_bio['choice'] = 0
    
    # Filling NaN with zeros
    bd_bio = bd_bio.fillna(0)
    
    # Adding available options
    for i in range(0,with_data+1):
        bd_bio['av_{}'.format(i)] = bd_bio.apply(lambda row: 1 if row['time_{}'.format(i)]>0 else 0, axis = 1) 
    
    # Adding available options
    for i in range(0,with_data2+1):
        bd_calc['av_{}'.format(i)] = bd_calc.apply(lambda row: 1 if row['time_{}'.format(i)]>0 else 0, axis = 1) 
    
    # Running Biogeme
    database = db.Database(e, bd_bio)
    globals().update(database.variables)

    atribs = ['time', 'n_nodes', 'angular_cost'] # distance
    betas = {}
    for b in atribs:
        auxBeta = Beta('B_{}'.format(b),0,-100,100,0) #vstart, vmin, vmax, isFix?
        betas.update({'B_{}'.format(b): auxBeta})

    V_choice = 0
    for v in atribs:
        V_choice += betas['B_{}'.format(v)]*database.variables[(v+'_{}').format(0)]

    V = {0: V_choice}    
    av = {0: database.variables['av_0']}
       
    for i in range(1,with_data+1):
        V_no_choice_aux = 0
        for v in atribs:
            V_no_choice_aux += betas['B_{}'.format(v)]*database.variables[(v+'_{}').format(i)]
        V.update({i:V_no_choice_aux})
        av.update({i: database.variables['av_{}'.format(i)]})

    logprob = models.loglogit(V, av, choice)
    biogeme = bio.BIOGEME(database,logprob)
    biogeme.modelName = e
    results = biogeme.estimate()
    print('    Biogeme finished')
    
    scenarios[e] = results.getBetaValues()
    bd_calc[['bio_choice','bio_max']] = bd_calc.apply(lambda row: bio_choice(row, scenarios[e], with_data2), axis = 1)
    bd_calc[['tes_choice','tes_max']] = bd_calc.apply(lambda row: tes_choice(row, with_data2), axis = 1)
    
    print('    Indicators ready')
    
    bd_calc['VAR_tes'] = bd_calc.apply(lambda row: row['tes_max']*(1-row['tes_max']), axis =1)
    bd_calc['VAR_bio'] = bd_calc.apply(lambda row: row['bio_max']*(1-row['bio_max']), axis =1)
    
    FPR_tes = len(bd_calc.loc[bd_calc['tes_choice']==0])/bd_calc['tes_choice'].count()
    FPR_bio = len(bd_calc.loc[bd_calc['bio_choice']==0])/bd_calc['bio_choice'].count()
    ER_tes = bd_calc['tes_max'].sum()/bd_calc['tes_choice'].count()
    ER_bio = bd_calc['bio_max'].sum()/bd_calc['bio_choice'].count()
    DS_tes = bd_calc['VAR_tes'].sum()**(1/2)/bd_calc['tes_choice'].count()
    DS_bio = bd_calc['VAR_bio'].sum()**(1/2)/bd_calc['bio_choice'].count()
    
    obs = bd_calc['bio_choice'].count()
    
    indicator[e] = {'FPR_BIO': FPR_bio, 
                    'LIM_INF_BIO': ER_bio-DS_bio*1.96, 
                    'LIM_SUP_BIO': ER_bio+DS_bio*1.96, 
                    'FPR_TES': FPR_tes, 
                    'LIM_INF_TES': ER_tes-DS_tes*1.96, 
                    'LIM_SUP_TES': ER_tes+DS_tes*1.96, 
                    'OBS': obs}

In [ ]:
summary = pd.DataFrame(indicator)
summary = summary.T
summary = summary.loc[:,['OBS', 'FPR_BIO', 'LIM_INF_BIO', 'LIM_SUP_BIO', 'FPR_TES', 'LIM_INF_TES', 'LIM_SUP_TES']]
summary.to_csv('summary.csv', index = False)

In [ ]:
summary.plot(kind = 'bar', y=['FPR_BIO', 'FPR_TES'], figsize=(10,5), grid=True)